In [1]:
import os
 
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["HF_DATASETS_CACHE"] = "/workspace/cache"
os.environ["TRANSFORMERS_CACHE"] = "/workspace/cache"

In [3]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("chompk/tydiqa-goldp-th")

Reusing dataset tydiqa-goldp-th (/workspace/cache/chompk___tydiqa-goldp-th/th/1.1.0/3bcb52f1a9a39ac616d6356b941e9eb8096b91a5850568245edd0bdf35388ac9)


  0%|          | 0/2 [00:00<?, ?it/s]

# dataset prepare

In [9]:
all_doc = set(dataset['validation']['context'])
all_doc = {c:i for i,c in enumerate(all_doc)}

question_contextid_context = []
for item in dataset['validation']:
    question = item['question']
    doc = item['context']
    question_contextid_context.append([all_doc[doc],question])
    
df_question = pd.DataFrame(question_contextid_context, columns =['doc_id','question'])
df_document = pd.DataFrame(zip(list(all_doc.values()),list(all_doc.keys())), columns =['doc_id','document'])

# test model

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [6]:
def sim_search(question_encoded,doc_encoded):
    query_map = np.full(doc_encoded.shape, question_encoded)
    sim_score = np.array([*map(np.inner,query_map,doc_encoded)])
    return np.argsort(sim_score)[::-1]

def evaluate(question_id,question_all,context_id,context_all,mrr_rank=10,status=True):
    top_1 = 0; top_5 = 0; top_10 = 0;
    mrr_score = 0
    context_id = np.array(context_id)
    sim_score = np.inner(question_all,context_all)
    status_bar = enumerate(sim_score)
    for idx,sim in status_bar:
        index = np.argsort(sim)[::-1]
        index_edit = [context_id[x] for x in index]
        idx_search = list(index_edit).index(question_id[idx])
        if idx_search == 0:
            top_1+=1
            top_5+=1
            top_10+=1
        elif idx_search < 5:
            top_5+=1
            top_10+=1
        elif idx_search < 10:
            top_10+=1  
        if idx_search < mrr_rank:
            mrr_score += (1/(idx_search+1))
    mrr_score/=len(question_all)
    return top_1,top_5,top_10,mrr_score

In [7]:
model_name = 'mrp/simcse-model-roberta-base-thai' # mrp/simcse-model-distil-m-bert
model = SentenceTransformer(model_name)

In [10]:
doc_context_id = df_document['doc_id'].to_list()    
doc_context_encoded = model.encode(df_document['document'].to_list(),convert_to_numpy=True,normalize_embeddings=True)

question_id = df_question['doc_id'].to_list()
questions = model.encode(df_question['question'].to_list(),convert_to_numpy=True,normalize_embeddings=True)

top_1,top_5,top_10,mrr = evaluate(question_id,questions,doc_context_id,doc_context_encoded)

print(f'{model_name}')
precision = top_1 / len(questions)
print(f"Traninng Score P@1: {precision:.3f}")
precision = top_5 / len(questions)
print(f"Traninng Score P@5: {precision:.3f}")
precision = top_10 / len(questions)
print(f"Traninng Score P@10: {precision:.3f}")
print(f"Mrr score:{mrr:.3f}")

mrp/simcse-model-roberta-base-thai
Traninng Score P@1: 0.526
Traninng Score P@5: 0.705
Traninng Score P@10: 0.763
Mrr score:0.602
